In [1]:
import numpy as np
import pandas as pd

In [2]:
crimes_df = pd.read_excel('Data/CrimesData.xlsx', sheet_name='2012')

In [3]:
titanic_df = pd.read_csv('Data/TitanicDataset.csv')

In [4]:
sales_df = pd.read_csv('Data/SalesData.csv', parse_dates=['OrderDate'])

In [5]:
import sqlite3
connection = sqlite3.connect('Data/chinook.db')
chinook_df = pd.read_sql_query('SELECT * FROM albums',con=connection)

### 1. Знайти кількість злочинів, що сталися у грудні 2012 року в інтервалі між першою годиною ночі та п&#39;ятою ранку (CrimesData).

In [6]:
crimes_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,01/01/2012,00:00:00,5C,NaN,I,NaN,426.0,NORTHEASTERN,NaN,NaN,1
1,01/01/2012,01:00:00,6C,0 E NORTH AV,I,NaN,141.0,CENTRAL,Charles North,"(39.3110400000, -76.6155600000)",1
2,01/01/2012,01:01:00,5A,1800 LETITIA AV,I,NaN,831.0,SOUTHWESTERN,Morrell Park,"(39.2641000000, -76.6571600000)",1
3,01/01/2012,01:01:00,6E,2100 AISQUITH ST,O,NaN,342.0,EASTERN,East Baltimore Midway,"(39.3139400000, -76.6015100000)",1
4,01/01/2012,01:10:00,4B,2200 EASTERN AV,O,KNIFE,212.0,SOUTHEASTERN,Upper Fells Point,"(39.2859300000, -76.5857200000)",1


In [7]:
crimes_df['CrimeDateTime'] = crimes_df['CrimeDate'].astype(str)+' '+crimes_df['CrimeTime'].astype(str)
crimes_df['CrimeDateTime'] = pd.to_datetime(crimes_df['CrimeDateTime'])

In [8]:
crimes_df['Year']= crimes_df['CrimeDateTime'].dt.year
crimes_df['Month']= crimes_df['CrimeDateTime'].dt.month
crimes_df['Hour']= crimes_df['CrimeDateTime'].dt.hour
crimes_df = crimes_df[crimes_df['Year']==2012]
crimes_df = crimes_df[crimes_df['Month']==1]
crimes_df.drop(['CrimeDate', 'CrimeTime', 'Year', 'Month'], axis=1, inplace=True)

In [9]:
print('Number of crimes in wanted period:')
len(crimes_df[(crimes_df['Hour']>=1)&(crimes_df['Hour']<5)])

Number of crimes in wanted period:


45

### 2. Знайти середню, мінімальну і максимальну вартість квитка в залежності від місця посадки і класу каюти (Титанік).

In [10]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
titanic_df.groupby(by=['Embarked','Pclass']).agg(['mean','min','max'])['Fare']

mean      min       max
Embarked Pclass                               
C        1       104.718529  26.5500  512.3292
         2        25.358335  12.0000   41.5792
         3        11.214083   4.0125   22.3583
Q        1        90.000000  90.0000   90.0000
         2        12.350000  12.3500   12.3500
         3        11.183393   6.7500   29.1250
S        1        70.364862   0.0000  263.0000
         2        20.327439   0.0000   73.5000
         3        14.644083   0.0000   69.5500

### 3. Найти загальну суму продажу по місяцях 2019-го року; вивести в форматі місяця - сума в порядку збільшення суми (Sales).

In [12]:
sales_df.head()

,OrderDate,Region,Rep,Item,Units,Unit Cost,Total
0,2019-01-06,East,Jones,Pencil,95,1.99,189.05
1,2019-01-23,Central,Kivell,Binder,50,19.99,999.50
2,2019-02-09,Central,Jardine,Pencil,36,4.99,179.64
3,2019-02-26,Central,Gill,Pen,27,19.99,539.73
4,2019-03-15,West,Sorvino,Pencil,56,2.99,167.44


In [13]:
pd.to_datetime(sales_df['OrderDate']);

In [14]:
sales_df['Month'] = sales_df['OrderDate'].dt.month
sales_df['Year'] = sales_df['OrderDate'].dt.year
sales_df = sales_df[sales_df['Year']==2019]

In [15]:
sales_byTotal = sales_df[['Month', 'Total']]
sales_byTotal.set_index('Month',inplace=True)
sales_byTotal.groupby(by=['Month']).sum().sort_values(by=['Total'])

,Total
Month,
3,167.44
8,174.65
4,448.65
9,505.84
5,512.78
2,719.37
11,778.89
10,827.08
6,988.50


### 4. Знайти всіх виконавців, у яких понад 10 альбомів (chinook/albums).

In [16]:
chinook_df.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [17]:
more_than_10 = pd.DataFrame(chinook_df.groupby(by=['ArtistId'])['Title'].count()>10)
more_than_10[more_than_10['Title']==True]

,Title
ArtistId,
22,True
58,True
90,True


### 5. Знайти всіх родичів по прізвищу. В результаті повинна вийти таблиця з двома колонками. У першій колонці повне ім&#39;я пасажира, у другій його родичів на борту(вважати родичами пасажирів, у яких збігається прізвище) (Titanic).

In [18]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
titanic_df['Surname'] = titanic_df['Name'].apply(lambda x: x.split(',')[0])
titanic_df = titanic_df[['Surname', 'Name']]

In [20]:
pd.DataFrame(titanic_df.groupby(by=['Surname','Name']).count())

Empty DataFrame
Columns: []
Index: [(Abbing, Abbing, Mr. Anthony), (Abbott, Abbott, Mr. Rossmore Edward), (Abbott, Abbott, Mrs. Stanton (Rosa Hunt)), (Abelson, Abelson, Mr. Samuel), (Abelson, Abelson, Mrs. Samuel (Hannah Wizosky)), (Adahl, Adahl, Mr. Mauritz Nils Martin), (Adams, Adams, Mr. John), (Ahlin, Ahlin, Mrs. Johan (Johanna Persdotter Larsson)), (Aks, Aks, Mrs. Sam (Leah Rosen)), (Albimona, Albimona, Mr. Nassef Cassem), (Alexander, Alexander, Mr. William), (Alhomaki, Alhomaki, Mr. Ilmari Rudolf), (Ali, Ali, Mr. Ahmed), (Ali, Ali, Mr. William), (Allen, Allen, Miss. Elisabeth Walton), (Allen, Allen, Mr. William Henry), (Allison, Allison, Master. Hudson Trevor), (Allison, Allison, Miss. Helen Loraine), (Allison, Allison, Mrs. Hudson J C (Bessie Waldo Daniels)), (Allum, Allum, Mr. Owen George), (Andersen-Jensen, Andersen-Jensen, Miss. Carla Christine Nielsine), (Anderson, Anderson, Mr. Harry), (Andersson, Andersson, Master. Sigvard Harald Elias), (Andersson, Andersson, Miss. Ebba Iris Alfrida), (Andersson, Andersson, Miss. Ellis Anna Maria), (Andersson, Andersson, Miss. Erna Alexandra), (Andersson, Andersson, Miss. Ingeborg Constanzia), (Andersson, Andersson, Miss. Sigrid Elisabeth), (Andersson, Andersson, Mr. Anders Johan), (Andersson, Andersson, Mr. August Edvard ("Wennerstrom")), (Andersson, Andersson, Mrs. Anders Johan (Alfrida Konstantia Brogren)), (Andreasson, Andreasson, Mr. Paul Edvin), (Andrew, Andrew, Mr. Edgardo Samuel), (Andrews, Andrews, Miss. Kornelia Theodosia), (Andrews, Andrews, Mr. Thomas Jr), (Angle, Angle, Mrs. William A (Florence "Mary" Agnes Hughes)), (Appleton, Appleton, Mrs. Edward Dale (Charlotte Lamson)), (Arnold-Franchi, Arnold-Franchi, Mr. Josef), (Arnold-Franchi, Arnold-Franchi, Mrs. Josef (Josefine Franchi)), (Artagaveytia, Artagaveytia, Mr. Ramon), (Asim, Asim, Mr. Adola), (Asplund, Asplund, Master. Clarence Gustaf Hugo), (Asplund, Asplund, Master. Edvin Rojj Felix), (Asplund, Asplund, Miss. Lillian Gertrud), (Asplund, Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)), (Astor, Astor, Mrs. John Jacob (Madeleine Talmadge Force)), (Attalah, Attalah, Miss. Malake), (Attalah, Attalah, Mr. Sleiman), (Aubart, Aubart, Mme. Leontine Pauline), (Augustsson, Augustsson, Mr. Albert), (Ayoub, Ayoub, Miss. Banoura), (Backstrom, Backstrom, Mr. Karl Alfred), (Backstrom, Backstrom, Mrs. Karl Alfred (Maria Mathilda Gustafsson)), (Baclini, Baclini, Miss. Eugenie), (Baclini, Baclini, Miss. Helene Barbara), (Baclini, Baclini, Miss. Marie Catherine), (Baclini, Baclini, Mrs. Solomon (Latifa Qurban)), (Badt, Badt, Mr. Mohamed), (Bailey, Bailey, Mr. Percy Andrew), (Balkic, Balkic, Mr. Cerin), (Ball, Ball, Mrs. (Ada E Hall)), (Banfield, Banfield, Mr. Frederick James), (Barah, Barah, Mr. Hanna Assi), (Barbara, Barbara, Miss. Saiide), (Barbara, Barbara, Mrs. (Catherine David)), (Barber, Barber, Miss. Ellen "Nellie"), (Barkworth, Barkworth, Mr. Algernon Henry Wilson), (Barton, Barton, Mr. David John), (Bateman, Bateman, Rev. Robert James), (Baumann, Baumann, Mr. John D), (Baxter, Baxter, Mr. Quigg Edmond), (Baxter, Baxter, Mrs. James (Helene DeLaudeniere Chaput)), (Bazzani, Bazzani, Miss. Albina), (Beane, Beane, Mr. Edward), (Beane, Beane, Mrs. Edward (Ethel Clarke)), (Beavan, Beavan, Mr. William Thomas), (Becker, Becker, Master. Richard F), (Becker, Becker, Miss. Marion Louise), (Beckwith, Beckwith, Mr. Richard Leonard), (Beckwith, Beckwith, Mrs. Richard Leonard (Sallie Monypeny)), (Beesley, Beesley, Mr. Lawrence), (Behr, Behr, Mr. Karl Howell), (Bengtsson, Bengtsson, Mr. John Viktor), (Berglund, Berglund, Mr. Karl Ivar Sven), (Berriman, Berriman, Mr. William John), (Betros, Betros, Mr. Tannous), (Bidois, Bidois, Miss. Rosalie), (Bing, Bing, Mr. Lee), (Birkeland, Birkeland, Mr. Hans Martin Monsen), (Bishop, Bishop, Mr. Dickinson H), (Bishop, Bishop, Mrs. Dickinson H (Helen Walton)), (Bissette, Bissette, Miss. Amelia), (Bjornstrom-Steffansson, Bjornstrom-Steffansson, Mr. Mauritz Hakan), (Blackwell, Blackwell, Mr.